## **LArCV file-making for ND-LAr and its prototypes on NERSC**

### **ND-LAr and protoypes workflow**
<figure>
<img src="assets/ND_workflow.png">
<figcaption align = "center"> ND software chain </figcaption>
</figure>

### **File locations on NERSC**
`/global/cfs/cdirs/dune/www/data/`

#### Create a data directory in your scratch area since the project area is not accessible from GPU nodes `/pscratch/sd`
`mkdir data_files`

### **1. Detector simulation (larndsim)**
##### This installation takes more time and running it requires GPU
##### The workshop software directory does not work for this. You should use `/pscratch/sd/`. Alternatively, you can just skip this step and use the existing simulation files in
`/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun6.5_1E19_RHC/MiniRun6.5_1E19_RHC.larnd/LARNDSIM/000000/`

##### Make sure you have GPU

`salloc --nodes 1 --qos shared_interactive --time 00:30:00 --constraint gpu --gpus 1 --account=m5252 --image=deeplearnphysics/larcv2:ub2204-cu121-torch251-larndsim shifter /bin/bash`

##### Git clone and install
```
git clone https://github.com/DUNE/larnd-sim.git
cd larnd-sim

```
##### For both the required dependencies and the optional packages to use lar_runner.py, do:

`pip install .[runner]`

##### lar_runner.py is a wrapper script to run larnd-sim with and without profiling and uses a YAML configuration file
##### make sure the installation path is linked to access lar_runner directly

`export PATH="$HOME/.local/bin:$PATH"`

##### Now you can produce a file using the default yaml config (2x2)

`lar_runner.py -y cli/runner.yaml run -c 2x2 -i /global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun6.5_1E19_RHC/MiniRun6.5_1E19_RHC.convert2h5/EDEPSIM_H5/0000000/MiniRun6.5_1E19_RHC.convert2h5.0000000.EDEPSIM.hdf5 -o  /global/cfs/cdirs/m5252/$USER/MiniRun6.5_1E19_RHC.larndsim.0000000.LARNDSIM.hdf5 -n 10`

### **2. Event building and detector calibration (ndlar-flow)**
```
cd ..
git clone https://github.com/DUNE/ndlar_flow.git
cd ndlar_flow
pip install .
```

##### Make your own sh script based on the workflows you want
##### example (run_flow.sh)
```

inFile=/global/cfs/cdirs/m5252/$USER/MiniRun6.5_1E19_RHC.larndsim.0000000.LARNDSIM.hdf5
outFile=/global/cfs/cdirs/m5252/$USER/MiniRun6.5_1E19_RHC.flow.0000000.FLOW.hdf5
workflow1='yamls/proto_nd_flow/workflows/charge/charge_event_building_mc.yaml'
workflow2='yamls/proto_nd_flow/workflows/charge/charge_event_reconstruction_mc.yaml'
workflow3='yamls/proto_nd_flow/workflows/combined/combined_reconstruction_mc.yaml'
workflow4='yamls/proto_nd_flow/workflows/charge/prompt_calibration_mc.yaml'
workflow5='yamls/proto_nd_flow/workflows/charge/final_calibration_mc.yaml'
workflow6='yamls/proto_nd_flow/workflows/light/light_event_building_mc.yaml'
workflow7='yamls/proto_nd_flow/workflows/light/light_event_reconstruction_mc.yaml'
workflow8='yamls/proto_nd_flow/workflows/charge/charge_light_assoc_mc.yaml'
h5flow $opts -c $workflow1 $workflow2 $workflow3 $workflow4 $workflow5 \
     -i "$inFile" -o "$outFile"
h5flow $opts -c $workflow6 $workflow7\
    -i "$inFile" -o "$outFile"
h5flow $opts -c $workflow8\
    -i "$outFile" -o "$outFile"

```

##### Don't forget to make it executable `chmod +x run_flow.sh`
##### Before running flow, make sure the pixel layouts are present. This is typically done running `ndlar_flow/scripts/proto_nd_scripts/get_proto_nd_input.sh`. 
##### In order to avoid complaints from curl, you can simply copy the layouts from larndsim:

`cp ../larnd-sim/larndsim/pixel_layouts/multi* data/proto_nd_flow/`

##### Run flow! `./run_flow.sh`

### **3. Data parsing for SPINE (flow2supera)**
##### Flow2supera is esentially a data parsing software that converts the 2x2 flow files into LArCV format (Liquid Argon Computer Vision, specific data structure for SPINE)
##### Flow2supera software is dependent on LArCV, SuperaAtomic (label maker), larndsim, h5flow which is already installed on s3df and and nersc 

##### You can use an sh script, copy paste the lines below to speed up the following installtions
```
cd ..
git clone https://github.com/DeepLearnPhysics/SuperaAtomic.git
cd SuperaAtomic
pip3 install .

cd ..
git clone https://github.com/DeepLearnPhysics/flow2supera.git
cd flow2supera
git checkout develop
pip3 install .

```

##### Run flow2supera to produce a larcv file! (look at configs directory for different options)
##### You can also specify number of events using -n or skip events using -s

`run_flow2supera.py -o /global/cfs/cdirs/m5252/$USER/MiniRun6.5_1E19_RHC.larcv.0000000.LARCV.root -c 2x2 /global/cfs/cdirs/m5252/$USER/MiniRun6.5_1E19_RHC.flow.0000000.FLOW.hdf5`

<figure>
<img src="assets/spongebob.jpg" width="500">
</figure>